In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
resultsData = pd.read_csv("/content/drive/MyDrive/NLP/taskCResults.csv",encoding='iso-8859-1')

In [ ]:
resultsData

,source_text,target_text,generated_stance_text
0,#Breaking: Pentagon releases video of the âm...,@TODAYshow \n\nBig expensive payload to kill 3...,@Domestic_market oh my god
1,#Breaking: Pentagon releases video of the âm...,$MENTION$ \n\nBig expensive payload to kill 36...,$mention$ Big expensive payload to kill 36 peo...
2,#Breaking: Pentagon releases video of the âm...,@TODAYshow How many ISIS did it kill?,@Domestic_market oh my god
3,#Breaking: Pentagon releases video of the âm...,$MENTION$ How many ISIS did it kill? #Breaking...,$mention$ How many ISIS did it kill #Breaking:...
4,#Breaking: Pentagon releases video of the âm...,@TODAYshow @HallieJackson The mother... give m...,@Domestic_market oh my god
...,...,...,...
2091,"I've been searching, and can't find a single c...","Yeah, they completely ignored my post. Even th...",sometimes you can't win an argument. :-) Ameri...
2092,"I've been searching, and can't find a single c...",If Obama had admitted his birth certificate wa...,.@obamacare did he actually admit that his bir...
2093,"I've been searching, and can't find a single c...",Exactly. If Obama had admitted his birth certi...,did Obama actually admit that his birth certif...
2094,"I've been searching, and can't find a single c...",Doesn't mean it's not fake. You idiots voted a...,.@obamacare did he actually admit that his bir...


In [ ]:
evaluation_text=resultsData.filter(items=['target_text','generated_stance_text']).values.tolist()

In [ ]:
cosine_similarities=[]
for corpus in evaluation_text:
  count_vect = CountVectorizer()
  X_train_counts = count_vect.fit_transform(corpus)

  pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names_out(),index=['target_text','generated_stance_text'])

  vectorizer = TfidfVectorizer()
  trsfm=vectorizer.fit_transform(corpus)
  pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names_out(),index=['target_text','generated_stance_text'])
  cosine_similarities.append(cosine_similarity(trsfm[0:1], trsfm))


In [ ]:
cosine_similarities

In [ ]:
cosine_data=[]
for item in cosine_similarities:
  cosine_data.append(item.tolist()[0][1])
  

In [ ]:
cosine_data

In [ ]:
import statistics

In [ ]:
print ("Mean:",statistics.mean(cosine_data))
print ("Median",statistics.median(cosine_data))
print ("Standard deviation:",statistics.stdev(cosine_data))
print ("Variance:",statistics.variance(cosine_data))

Mean: 0.4040857226539669
Median 0.19257350996870876
Standard deviation: 0.39628707049681233
Variance: 0.1570434422429455


In [ ]:
import nltk


In [ ]:
bleu_scores=[]
for corpus in evaluation_text:
  hypothesis=corpus[1].split()
  reference=corpus[0].split()
  #there may be several references
  BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
  bleu_scores.append(BLEUscore)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
print ("Mean:",statistics.mean(bleu_scores))
print ("Median",statistics.median(bleu_scores))
print ("Standard deviation:",statistics.stdev(bleu_scores))
print ("Variance:",statistics.variance(bleu_scores))

Mean: 0.36161646663496333
Median 0.4066246764097428
Standard deviation: 0.3434362173134457
Variance: 0.11794843536256829


In [ ]:
bleu_scores

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_metric
bertscore = load_metric("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [ ]:
results

{'f1': [1.0, 1.0],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.11(hug_trans=4.19.0)',
 'precision': [1.0, 1.0],
 'recall': [1.0, 1.0]}

In [ ]:
all_references=[]
all_hypothesis=[]
for references,hypothesis in evaluation_text:
    all_references.append(references)
    all_hypothesis.append(hypothesis)

In [ ]:
from datasets import load_metric
bertscore = load_metric("bertscore")
predictions = all_hypothesis
references = all_references
results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [ ]:
print ("Hashcode:",results['hashcode'])
print ("Mean of BERT F1:",statistics.mean(results['f1']))
print ("Mean of BERT Precision:",statistics.mean(results['precision']))
print ("Mean of BERT Recall:",statistics.mean(results['recall']))


Hashcode: roberta-large_L17_no-idf_version=0.3.11(hug_trans=4.19.0)
Mean of BERT F1: 0.8543797824958353
Mean of BERT Precision: 0.8467467028277976
Mean of BERT Recall: 0.8635648247565477


In [ ]:
!pip install rouge_score
rouge = load_metric('rouge')
predictions = all_hypothesis
references = all_references
results = rouge.compute(predictions=predictions,references=references)
print(list(results.keys()))

['rouge1', 'rouge2', 'rougeL', 'rougeLsum']


In [ ]:
results

{'rouge1': AggregateScore(low=Score(precision=0.4251271271125035, recall=0.4373630994118866, fmeasure=0.4161240447328763), mid=Score(precision=0.441869178964449, recall=0.45561177213253007, fmeasure=0.43370709185583556), high=Score(precision=0.4586129726488238, recall=0.47247876916824005, fmeasure=0.45029000097689753)),
 'rouge2': AggregateScore(low=Score(precision=0.3819559024919208, recall=0.3999018996642332, fmeasure=0.3807335663192744), mid=Score(precision=0.39959570841004133, recall=0.41828578882066375, fmeasure=0.39844333431819157), high=Score(precision=0.41756983963119615, recall=0.4369868128739215, fmeasure=0.4163796071175468)),
 'rougeL': AggregateScore(low=Score(precision=0.4164099911290084, recall=0.4313838334980544, fmeasure=0.4102650993319388), mid=Score(precision=0.43291338466662976, recall=0.449612498737404, fmeasure=0.42740240715039857), high=Score(precision=0.4498241332041679, recall=0.4668836699760964, fmeasure=0.4446337746276702)),
 'rougeLsum': AggregateScore(low=Sc